In [25]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os


##

In [26]:
import glob
PATH = "/Users/raminduwalgama/Documents/Projects/OneID/icao-guidelines-server/detectors/geometric_tests/"

all_images = glob.glob("../../dataset/valid/*.JPG")

In [32]:
from datetime import datetime
import pandas as pd

def logger(image, params, path="./"):
    time_str = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    df_headers = pd.DataFrame({
        'log_time': [time_str],
        'image': [image],
        'My': params['My'],
        'MyTrue': params['MyTrue'],
        'Mx': params['Mx'],
        'MxTrue': params['MxTrue'],
        'CC': params['CC'],
        'CCTrue': params['CCTrue'],
        'DD': params['DD'],
        'DDTrue': params['DDTrue'],
        'valid': params['valid']
    })
    if not os.path.isfile(f'{path}geometric_logs.csv'):
        df_headers.to_csv(f'{path}geometric_logs.csv', mode='w', index=False)
    else:
        df_headers.to_csv(f'{path}geometric_logs.csv', mode='a', index=False, header=False)

In [28]:
def get_face_landmarks(image_path):
    model_path = os.path.join(PATH, "../utilities/face_landmarker.task")
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.FaceLandmarkerOptions(
        base_options=base_options,
        output_face_blendshapes=True,
        output_facial_transformation_matrixes=True,
        num_faces=1,
    )
    detector = vision.FaceLandmarker.create_from_options(options)
    image = mp.Image.create_from_file(image_path)
    detection_result = detector.detect(image)
    face_landmarks_list = detection_result.face_landmarks
    face_landmarks = face_landmarks_list[0]
    return face_landmarks


In [29]:
def geometric_test_vertical_position(face_landmarks):
    My = (face_landmarks[168].y + face_landmarks[6].y) / 2
    return My, 0.3 <= My <= 0.5


# Function to check the horizontal position
def geometric_test_horizontal_position(face_landmarks):
    Mx = (face_landmarks[234].x + face_landmarks[454].x) / 2
    return Mx, 0.45 <= Mx <= 0.55


# Function to check the head image width ratio
def geometric_test_head_image_width_ratio(face_landmarks):
    dy = face_landmarks[454].y - face_landmarks[234].y
    dx = face_landmarks[454].x - face_landmarks[234].x
    CC = ((dy**2) + (dx**2)) ** 0.5
    return CC, 0.5 <= CC <= 0.75


# Function to check the head image height ratio
def geometric_test_head_image_height_ratio(face_landmarks):
    dy = face_landmarks[152].y - face_landmarks[10].y
    dx = face_landmarks[152].x - face_landmarks[10].x
    DD = ((dy**2) + (dx**2)) ** 0.5
    return DD, 0.6 <= DD <= 0.9


In [34]:
def get_geometric_details(image_path):
    face_landmarks = get_face_landmarks(image_path)
    # Geometric test - Vertical position
    vp, vp_check = geometric_test_vertical_position(face_landmarks)
    # Geometric test - Horizontal position
    hp, hp_check = geometric_test_horizontal_position(face_landmarks)
    # Geometric test - Head image width ratio
    hiwr, hiwr_check = geometric_test_head_image_width_ratio(face_landmarks)
    # Geometric test - Head image height ratio
    hihr, hihr_check = geometric_test_head_image_height_ratio(face_landmarks)

    return {
        'My': vp,
        'MyTrue': vp_check,
        'Mx': hp,
        'MxTrue': hp_check,
        'CC': hiwr,
        'CCTrue': hiwr_check,
        'DD': hihr,
        'DDTrue': hihr_check,
        'valid': vp_check and hp_check and hiwr_check and hihr_check
    }

In [35]:
from datetime import datetime

start_time = datetime.now()

for j in range(0, int(len(all_images))):
    image_name = all_images[j]
    data = get_geometric_details(image_name)
    logger(image_name,data)

print("------------------COMPLETED------------------")

end_time = datetime.now()
print('Unit: seconds')
print('Duration: {}'.format(end_time - start_time))
print('Duration per image: {}'.format((end_time - start_time)/len(all_images)))

W20230930 11:20:44.891278 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230930 11:20:44.940820 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230930 11:20:44.975958 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230930 11:20:45.008100 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230930 11:20:45.041561 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230930 11:20:45.074455 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230930 11:20:45.199697 1550298 face_l

------------------COMPLETED------------------
Unit: seconds
Duration: 0:00:16.798323
Duration per image: 0:00:00.033597


W20230930 11:21:01.596362 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230930 11:21:01.627183 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230930 11:21:01.657202 1550298 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.


In [37]:
df = pd.read_csv('geometric_logs.csv')
df.describe()

,My,Mx,CC,DD
count,500.000000,500.000000,500.000000,500.000000
mean,0.440799,0.502904,0.572014,0.517755
std,0.023614,0.009329,0.038211,0.036722
min,0.336021,0.476456,0.455072,0.383389
25%,0.428273,0.497254,0.547017,0.496129
50%,0.442576,0.502759,0.572249,0.518407
75%,0.454766,0.508782,0.597775,0.541900
max,0.501052,0.539011,0.677985,0.630345
